## * 2019 3rd ML month with KaKR
+ 자동차 이미지 데이터셋을 활용한 자동차 차종 분류
+ 대회 링크: https://www.kaggle.com/c/2019-3rd-ml-month-with-kakr

<img src="./images/main_img.png" width="70%" height="70%">

## < 필요 모듈 임포트 >

In [ ]:
import gc
import os
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import cv2
import PIL
from PIL import ImageOps, ImageFilter, ImageDraw

from keras.callbacks import Callback
from keras import backend
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D, BatchNormalization, Input
from keras.optimizers import Adam, SGD, Nadam
from keras.metrics import categorical_accuracy
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from keras import backend as K
warnings.filterwarnings(action='ignore')

# EfficientNet Download
!pip install git+https://github.com/qubvel/efficientnet
from efficientnet.keras import EfficientNetB3

K.image_data_format()

## < 데이터 확인 >

In [1]:
def make_crop_df(df):
    dfcopy = df.copy(deep = True)
    dfcopy['img_file'] = dfcopy['img_file'].apply(lambda x : 'crop' + x)
    return pd.concat([df, dfcopy], axis = 0)

In [ ]:
# 모델 저장 및 로드 경로
model_path = './model/'
if(not os.path.exists(model_path)):
    os.mkdir(model_path)

# 데이터 경로 확인
img_path = os.listdir('../input')
print(img_path)

CROP_PATH = '../input/' + img_path[2]
DATA_PATH = '../input/' + img_path[1]

# image folder path
TRAIN_IMG_PATH = CROP_PATH +'/train_crop/'
TEST_IMG_PATH = CROP_PATH +'/test_crop/'

# read csv
df_train = pd.read_csv(DATA_PATH + '/train.csv')
df_test = pd.read_csv(DATA_PATH + '/test.csv')

# Centered Crop된 이미지를 사용하기 위해 경로를 crop 폴더로 변경해줌
df_train = make_crop_df(df_train)
df_train["class"] = df_train["class"].astype('str')

df_train = df_train[['img_file', 'class']]
df_test = df_test[['img_file']]

## < EDA >
+ 클래스 개수는 특정 유형이 많거나 적은 경우가 1~2개 클래스 존재하지만,
+ 실험 결과에 큰 영향을 주지 않으므로 그대로 진행
+ 기존 noise가 포함된 이미지보다 crop된 이미지가 성능에 가장 큰 향상을 보여줌으로 이를 사용

In [ ]:
# 클래스 개수 확인
plt.figure(figsize=(15,6))
sns.countplot('class', data=df_train)
plt.show()

## < 이미지 제네레이터 정의 >
+ shear_range는 성능에 좋지 않음
+ 클래스 불균형으로 인해 class_weight를 추가했지만, 성능 향상을 보이지 못함

In [ ]:
# Parameter
# nb_train_samples = len(X_train)
# nb_validation_samples = len(X_val)
nb_test_samples = len(df_test)
batch_size = 32

# Define Generator config
# https://www.kaggle.com/kozistr/seedlings-densenet-161-48-public-lb-98-236
train_datagen = ImageDataGenerator(
    rotation_range = 60,
    brightness_range=[0.5, 1.5],
    width_shift_range=0.30,
    height_shift_range=0.30,
    horizontal_flip = True, 
    vertical_flip = False,
    zoom_range=0.25,
    fill_mode = 'nearest',
    rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0 :
        return (num_samples // batch_size) + 1
    else :
        return num_samples // batch_size

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(df_train['class']),
                                                 df_train['class'])

## < Metric 정의 >

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## < Model 정의 >
+ 최종 결과에서는 efficientNet, Incep_ResNet, Inception 등 다양한 모델을 학습
+ f1 지표가 가장 좋은 모델을 선택하는 것이 최종적으로 좋은 성능을 보여주었음

In [ ]:
# init params
lr = 2e-4

# 콜백 정의
def get_callback(model_path):
    callback_list = [
              ModelCheckpoint(filepath=model_path, monitor='val_f1_m',
                      verbose=1, save_best_only=True, mode = 'max'),
              ReduceLROnPlateau(monitor='val_f1_m',
                        factor=0.2,
                        patience=3,
                        min_lr=1e-7,
                        cooldown=1,
                        verbose=1, mode = 'max'),
              EarlyStopping(monitor = 'val_f1_m', patience = 5, mode = 'max')
              ]
    return callback_list

def get_model(base_model, input_size, train_session):
    base_model = base_model(weights='imagenet', input_shape=(input_size,input_size,3), include_top=False)

    inputs = Input(shape = (input_size, input_size, 3), name = 'input_1')
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, kernel_initializer='he_normal')(x)
    x = Dropout(0.3)(x)
    x = Activation('relu')(x)
    x = Dense(196, activation = 'softmax')(x)

    model = Model(inputs = inputs, outputs = x)
    if(train_session):
        nadam = Adam(lr = lr)
        model.compile(optimizer= nadam, loss='categorical_crossentropy', metrics=[categorical_accuracy,
                                                                              f1_m, precision_m, recall_m])
    return model

k_folds = 8
img_size = (299, 299)
skf = StratifiedKFold(k_folds, random_state = 2019)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=TEST_IMG_PATH,
    x_col='img_file',
    y_col=None,
    target_size= img_size,
    color_mode='rgb',
    class_mode=None,
    batch_size=batch_size,
    shuffle=False
)

## < Training >
+ 세션이 자주 끊기는 것은 학습에 가장 큰 문제점이었음
+ 별다른 방법이 없으므로 끊길 때마다 모니터링해주고, 다시 학습을 진행..
+ 간혹 OOM 문제가 뜨는데, gc.collect()는 효과가 없었고, clear_session()이 효과가 있는 것 같았음.
+ 단, clear_session()이 for-loop 안에서 동시에 실행되야하고, 다른 셀에서 실행된 경우 효과를 보여주지 않았음(Keras 내부 문제라고..)

In [ ]:
j = 1 # Session이 끊기는 경우가 있으므로, 연속된 학습을 위해 설정

img_size = (299, 299)
model_names = []
epochs = 100

# mandatory, MODEL name
BASE_MODEL = EfficientNetB3

for (train_index, valid_index) in skf.split(
    df_train['img_file'], 
    df_train['class']):
    
    # Session 끊기면 사용
#     if(j < 7):
#         j += 1
#         continue
#     elif(j == 8):
#         break

    traindf = df_train.iloc[train_index, :].reset_index()
    validdf = df_train.iloc[valid_index, :].reset_index()
    nb_train_samples = len(traindf)
    nb_validation_samples = len(validdf)

    print("=========================================")
    print("====== K Fold Validation step => %d/%d =======" % (j,k_folds))
    print("=========================================")

    # Make Generator
    train_generator_299 = train_datagen.flow_from_dataframe(
        dataframe=traindf, 
        directory=TRAIN_IMG_PATH,
        x_col = 'img_file',
        y_col = 'class',
        target_size = img_size,
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        seed=42
    )

    validation_generator_299 = val_datagen.flow_from_dataframe(
        dataframe=validdf, 
        directory=TRAIN_IMG_PATH,
        x_col = 'img_file',
        y_col = 'class',
        target_size = img_size,
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True
    )

    model_name = model_path + str(j) + '_EFF_segcrop.hdf5'
    model_names.append(model_name)
    new_model = get_model(BASE_MODEL, img_size[0], train_session=True)
    
    try:
        new_model.load_weights(model_name)
    except:
        pass

    history = new_model.fit_generator(
                        train_generator_299,
                        steps_per_epoch = get_steps(nb_train_samples, 32),
                        epochs=epochs,
                        validation_data = validation_generator_299,
                        validation_steps = get_steps(nb_validation_samples, 32),
                        callbacks =  get_callback(model_name),
                        class_weight = class_weights)
        
#     j+=1
    print(gc.collect()) # 효과 없음
    K.clear_session() # 가끔 효과 있음(?)

## < Conclusion >
+ 다양한 optimizer 실험에서 adam optimizer가 가장 좋은 성능
+ cosine annealing 등 실험해보았지만 오히려 학습 시간이 길어져 학습 결과를 확인하는 데 너무 오래걸림
+ 1e-4, 2e-4에서 고정된 lr로 시작하는 것이 효과적
+ 결과 제출에는 한 가지 모델보다 다른 구조의 모델을 학습하여 앙상블한 경우와 성능 차이가 크게 났음

## < Prediction >
+ 간단한 TTA(Test Time Augmentation)을 사용
+ EfficientNet, Xception, Incepres, Cutmix model을 앙상블함

In [ ]:
from keras.models import load_model

# MODEL folder path
EFF_PATH = os.path.join('../input', list_dir[5])
XCEPTION_PATH = os.path.join('../input', list_dir[1])
INCEPRES_PATH = os.path.join('../input', list_dir[4])
CUTMIX_PATH = os.path.join('../input', list_dir[1])

xception_model = ['_Xception_f1_8fold.hdf5', Xception]
eff_model = ['_EFF_f1_8fold.hdf5', EfficientNetB3]
incepres_model = ['_IncepRes_f1_8fold.hdf5', InceptionResNetV2]
cutmix_model = ['_cmm.hdf5', InceptionResNetV2]

model_list = [xception_model, eff_model, incepres_model]
# total predictions list
preds_list = []

TTA_STEPS = 5

for model_name, base_model in model_list:
    print(model_name)
    # prediction each fold
    img_size = 299
    predictions = []
    fold_num = 8 + 1
    
    # model_load_dir
    if(model_name == '_EFF_f1_8fold.hdf5'):
        model_load_dir = EFF_PATH
    elif(model_name == '_Xception_f1_8fold.hdf5'):
        model_load_dir = XCEPTION_PATH
    elif(model_name == '_IncepRes_f1_8fold.hdf5'):
        model_load_dir = INCEPRES_PATH
    elif(model_name == '_cmm.hdf5'):
        model_load_dir = CUTMIX_PATH

    for i in range(1, fold_num):
        model = get_model(base_model, img_size, train_session = False)
        # '..input/EFF_F1_8fold/i_EFF_f1_8fold.hdf5'
        model.load_weights(os.path.join(model_load_dir, str(i)) + model_name)
        # tta prediction list
        tta_preds = []
        for _ in range(TTA_STEPS):
            if(img_size == 224):
                test_generator_224.reset()
                pred = model.predict_generator(
                generator = test_generator_224, 
                steps = get_steps(nb_test_samples, batch_size),
                verbose = 1
                )
            else:
                test_generator_299.reset()
                pred = model.predict_generator(
                generator = test_generator_299, 
                steps = get_steps(nb_test_samples, batch_size),
                verbose = 1
                )
            tta_preds.append(pred) # (TTA_STEP, 6150, 196)
        tta_preds = np.mean(tta_preds, axis = 0) # (6150, 196)
        predictions.append(tta_preds) # (fold, 6150, 196)
        # for memory leaky
        del model
        for _ in range(10):
            gc.collect()
        K.clear_session()
    preds_list.append(np.mean(predictions, axis = 0))

## < Make Submission >
+ 성능 저하로 인해 cutmix 모델은 제외함
+ 가중은 0.5, 0.4 등 다양한 범위의 수치로 조합하여 사용해보았지만, (0.34, 0.34, 0.32)가 제일 좋은 성능을 보여주었음

In [ ]:
# agg prediction
preds = (preds_list[0] * 0.34) + (preds_list[1] * 0.34) + (preds_list[2] * 0.32)
preds_class_indices=np.argmax(preds, axis=1)
preds_labels = (train_generator_299.class_indices)
labels = dict((v,k) for k,v in preds_labels.items())
final_predictions = [labels[k] for k in preds_class_indices]

# make submission file
submission = pd.read_csv(os.path.join(CSV_PATH, 'sample_submission.csv'))
submission["class"] = final_predictions
submission.to_csv("submission.csv", index=False)
submission.head()